## Capture images

In [ ]:
import time
import cv2
import datetime as dt
from timelapse.utils import calculate_brightness, find_correct_exposure, get_properties, \
find_correct_exposure, find_best_focus, calculate_sharpness, init_cap_dict, end_cap
from notify_run import Notify
notify = Notify()

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s',
                    filename='../timelapse.log', 
                    filemode='w', 
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
logging.info('Started')

In [ ]:
MIN_BRIGHTNESS = 90
MAX_BRIGHTNESS = 95

In [ ]:
starttime = time.time()
notification_sent = False

properties = {'CV_CAP_PROP_FRAME_WIDTH':1280, 
              'CV_CAP_PROP_FRAME_HEIGHT':720, 
              'CV_CAP_PROP_EXPOSURE':15,
              'CV_CAP_PROP_AUTOFOCUS':0,
              'CV_CAP_PROP_FOCUS':0}

while True:
    logging.info('New iteration ----------')
    try:
        properties, n_iter = find_correct_exposure(properties, MIN_BRIGHTNESS, MAX_BRIGHTNESS,max_iter=75, factor=1.05)
        properties = find_best_focus(properties)

        # REINITIALIZE THE CAMERA
        cap = init_cap_dict(properties)

        # TAKE A PICTURE AND STORE IT
        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        ret,frame = cap.read()
        props_log = get_properties(cap)
        cv2.imwrite(f'../images/capture_{timestamp}.png',frame)
        end_cap(cap)
        logging.info(f'capture_{timestamp}.png ')
        logging.info(f'exposure: {properties["CV_CAP_PROP_EXPOSURE"]}, brightness: {calculate_brightness(frame):.2f}')
        logging.info(f'focus: {properties["CV_CAP_PROP_FOCUS"]}, sharpness: {calculate_sharpness(frame):.2f}')
        logging.info(f'props: {props_log}\n\n')
        
    except:
        if not notification_sent:
            notify.send('Timelapse has run into an error')
            notification_sent=True
        try:
            cap.release()
            cv2.destroyAllWindows()
        except:
            pass

        
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))